In [29]:
import geopandas as gpd
import pandas as pd
import os

In [8]:
# Add supplement data
df_supplement = pd.read_csv('network_supplement.dat', sep='\t', header=None, 
                            names=['origin', 'dest', 'capacity', 'length', 'fft', 'alpha', 'beta', 'speedLimit', 'LinkType', 'NMoto', 'NCar'])
# read TRTS4S_Y110_origin.csv
df_origin = gpd.read_file('TRTS4S_Y110_origin.csv')

sup_dict = {}
for _, row in df_origin.iterrows():
    origin = row["A"]
    dest = row["B"]
    level = int(row["LEVEL_110"]) or 0
    NMoto = row["NMT_110"] or 0
    if row["CLASS_110"] == 84:
        NCar = 1
    else:
        NCar = 0
    if (origin, dest) not in sup_dict:
        sup_dict[(origin, dest)] = {
            'LinkType': level, 'NMoto': NMoto, 'NCar': NCar
        }
    if (dest, origin) not in sup_dict:
        sup_dict[(dest, origin)] = {
            'LinkType': level, 'NMoto': NMoto, 'NCar': NCar
        }

print("Origin and destination pairs processed:", len(sup_dict))

Origin and destination pairs processed: 32999


In [28]:
for idx, row in df_supplement.iterrows():
    origin = str(int(row['origin']))
    dest = str(int(row['dest']))
    if (origin, dest) in sup_dict:
        df_supplement.loc[idx, 'LinkType'] = int(sup_dict[(origin, dest)]['LinkType'])
        df_supplement.loc[idx, 'NMoto'] = int(sup_dict[(origin, dest)]['NMoto'])
        df_supplement.loc[idx, 'NCar'] = int(sup_dict[(origin, dest)]['NCar'])

# Check for NaN values and convert to appropriate integer type
has_nan = (df_supplement['LinkType'].isna().any() or 
          df_supplement['NMoto'].isna().any() or 
          df_supplement['NCar'].isna().any())

if has_nan:
    # Use nullable integer type if there are NaN values
    df_supplement['LinkType'] = df_supplement['LinkType'].astype('Int64')
    df_supplement['NMoto'] = df_supplement['NMoto'].astype('Int64')
    df_supplement['NCar'] = df_supplement['NCar'].astype('Int64')
    print("Used Int64 (nullable integer) due to NaN values")
else:
    # Use regular integer type if no NaN values
    df_supplement['LinkType'] = df_supplement['LinkType'].astype('int64')
    df_supplement['NMoto'] = df_supplement['NMoto'].astype('int64')
    df_supplement['NCar'] = df_supplement['NCar'].astype('int64')
    print("Used int64 (regular integer) - no NaN values found")

df_supplement

Used int64 (regular integer) - no NaN values found


,origin,dest,capacity,length,fft,alpha,beta,speedLimit,LinkType,NMoto,NCar
0,14043,9070,770.0,0.29,0.54375,1.5554,3.829,0.0,6,0,0
1,9069,14043,770.0,0.15,0.28125,1.5554,3.829,0.0,6,0,0
2,1088,10930,9999.0,0.44,6.60000,0.0000,0.000,0.0,11,0,0
3,10930,1088,9999.0,0.44,6.60000,0.0000,0.000,0.0,11,0,0
4,774,79999,9999.0,14.50,217.50000,0.0000,0.000,0.0,11,0,0
...,...,...,...,...,...,...,...,...,...,...,...
3757,12145,767,9999.0,0.48,7.20000,0.0000,0.000,0.0,11,0,0
3758,773,69999,9999.0,5.88,88.20000,0.0000,0.000,0.0,11,0,0
3759,69999,773,9999.0,5.88,88.20000,0.0000,0.000,0.0,11,0,0
3760,776,69999,9999.0,31.61,474.15000,0.0000,0.000,0.0,11,0,0


In [30]:
# old network_supplement.dat becomes network_supplement_old.dat
os.rename('network_supplement.dat', 'network_supplement_old.dat')
# save the updated DataFrame to a new file
output_file = 'network_supplement.dat'
df_supplement.to_csv(output_file, index=False, sep='\t')